This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [22]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [23]:
contexts = redbiom.summarize.contexts()

In [ ]:
contexts.head()

At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [ ]:
contexts.sort_values('SamplesWithData', ascending=False).head()

For simplicity sake, let's take the first context. It's large, and the phylogeny associated with the operational taxonomic units is easy to get. We'll break down the meaning of the context name in a moment. In practice, you will _most likely_ want to use the Deblur data, however producing a reasonable tree from those data requies a slightly computationally expensive step, and I'm on my laptop right now with limited battery quite literally in the middle of nowhere on a bus in the Czech Republic.

In [ ]:
ctx = contexts.sort_values('SamplesWithData', ascending=False).iloc[0]['ContextName']

In [ ]:
ctx

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [ ]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [ ]:
for k, v in results.items():
    print(k, len(v))

As you can see, the sample sets are not very balanced. For the purposes of the mini-analysis, let's just grab the first 100 from each sample type.

In [ ]:
to_keep = []
for k, v in results.items():
    to_keep.extend(list(v)[:100])

In [ ]:
to_keep[:5]

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

In [ ]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep)

In [ ]:
biom_table

The `repr` output shows that we have roughly 13k OTUs (operational taxonomic units), and only 244 samples. What gives? We were supposed to get 300! Just because a sample has metadata does not mean it has sequence data. It is also possible that some of the samples haven't been run through the same bioinformatic processing (e.g., closed reference at 100nt).

More information on `biom` can be found [here](http://biom-format.org/). 

Let's play with the object for just a moment for familiarity.

In [ ]:
biom_table.head()

In [ ]:
print(biom_table.head(6))

When we print the table, it implicitly casts it to a dense matrix representation. One thing to note: the sample IDs look different than before, right? The way that the individual sequencing runs are denoted is the last number -- the sample ID is represented by "qiita_study_id.the_actual_id.the_sequencing_run_id". This is to ensure the identifier is "globally" unique.

The identifiers on the rows are the "Greengenes" IDs. Also called OTU IDs. Or Feature IDs.

So we have a table. Now let's get the sample metadata. This will come down as a pandas `DataFrame`.

In [ ]:
md, _ = redbiom.fetch.sample_metadata(to_keep, context=ctx)

In [ ]:
md.head()

In [ ]:
md.sample_type.value_counts()

Great! So we have a bunch of sample data, and some study variables! Now for the QIIMEing. What we'll need to do is pacakge these data into types that QIIME 2 understands. And in particular, we need to use the semantic type system. This is well documented on the QIIME 2 website, and I recommend reviewing there.

In [ ]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', biom_table)
md_ar = qiime2.Metadata(md.set_index('#SampleID'))

Ah! We need a tree! Since we're using Greengenes, we can just rely on the existing prebuilt tree from the reference. Let's get that.

In [ ]:
!wget ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/97_otus.tree

In [ ]:
tree_ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', '97_otus.tree')

Excellent. So next up, we need to rarefy these data. Rarefaction is a just randomly subsampling the samples without replacement in order to normalize for sequencing effort. There are a lot of ways to do this type of normalization. Rarefaction is dirty as you through out data, but it's pragmatic and tends to work pretty well. The issue rarefaction helps to ameliorate is that, the more you sequence the more life you'll observe. So if you don't do something to normalize sequencing effort across samples, you'll have a bad time.

Let's import all the plugins we'll be using.

In [ ]:
from qiime2.plugins import feature_table, diversity, emperor

In [ ]:
# rarefy to 1000 sequences per sample (yes, it's arbitrary)
rare_ar, = feature_table.actions.rarefy(table=table_ar, sampling_depth=1000)

In [ ]:
unweighted_unifrac_ar, = diversity.actions.beta_phylogenetic(table=rare_ar, 
                                                             phylogeny=tree_ar, 
                                                             metric='unweighted_unifrac')

So before we move on, we should discuss what this voodoo just did. Beta diversity is what ecologists call comparing how similar (or dissimilar) two samples are. For example, how similar is a forest in the Pacific Northwest to a forest in Maine? (i.e., beta diversity is a function which takes two vectors and returns a scaler). We perform this calculation over all pairs of samples though so that we can examine all of the sample relationships. This distance matrix gets large quick: in our case, it's already 244 by 244. 

Visualizing that matrix would suck. So, one thing we often do is principal *coordinates* analysis. It's very similar to principal *components* analysis, except that we can pass in our distance matrix.

In [ ]:
unweighted_unifrac_pcoa_ar, = diversity.actions.pcoa(unweighted_unifrac_ar)

Finally, let's actually view the coordinates. It's much more interesting to use the metadata as well :) Hint: color by body site.

In [ ]:
viz, = emperor.actions.plot(unweighted_unifrac_pcoa_ar, md_ar)

In [ ]:
viz